Hello
Below you will find a tutorial on how to use ATLAS_Parser - a tool made for working with ATLAS Open Data

In [ ]:
#Important imports
from src.parse_atlas import parser, schemas
from src.calculations import physics_calcs

In [ ]:
#Config for the tool
config = {
    "max_process_memory_mb": 8192,
    "max_chunk_size_bytes": 500_000_000,
    "max_threads": 3,
    "release_year": "2024",
    "file_limit": None
}

In [ ]:
#Initializing the parser with configuration
atlasparser = parser.ATLAS_Parser(
        max_process_memory_mb=config["max_process_memory_mb"],
        max_chunk_size_bytes=config["max_chunk_size_bytes"],
        max_threads=config["max_threads"]
        )

#Fetching file URIs for the specified release year
release_files_uris = atlasparser.fetch_records_ids(
    release_year=config["release_year"]
)

#Parsing files in chunks and processing events
for events_chunk in atlasparser.parse_files(
    files_ids=release_files_uris, 
    limit=config["file_limit"]
):
    #Here you can do stuff with events_chunk
    pass

In [ ]:
#A 
atlasparser = parser.ATLAS_Parser(
        max_process_memory_mb=config["max_process_memory_mb"],
        max_chunk_size_bytes=config["max_chunk_size_bytes"],
        max_threads=config["max_threads"]
        )

release_files_uris = atlasparser.fetch_records_ids(
    release_year=config["release_year"]
)

for events_chunk in atlasparser.parse_files(
    files_ids=release_files_uris, 
    limit=config["file_limit"]
):
        
    logger.info("Cutting events")
    cut_events = physics_calcs.filter_events_by_kinematics(
        events_chunk, config["kinematic_cuts"]
    )
    del events_chunk  

    logger.info("Filtering events")
    filtered_events = physics_calcs.filter_events_by_particle_counts(
        events=cut_events, 
        particle_counts=config["particle_counts"], 
        is_particle_counts_range=True
    ) 
    del cut_events

    logger.info("Flattening root")
    root_ready = atlasparser.flatten_for_root(filtered_events)
    del filtered_events

    logger.info("Saving events")
    atlasparser.save_events_as_root(root_ready, config["output_path"])        
    del root_ready

    gc.collect()  

In [ ]:
#MAIN GOAL OF THIS, FIGURE OUT IF TAU OR RHO ARE PRESENT
import uproot
from src.parse_atlas import parser, schemas

atlas = parser.ATLAS_Parser(1,1,1)
release_files_uris = atlas.fetch_records_ids(
    release_year='2024')
import random
random.shuffle(release_files_uris)
keys = []

for uri in release_files_uris[:50]:
    with uproot.open(uri) as file:
        tree = file["CollectionTree"]
        keys.extend(tree.keys())
        print(f"Processed {uri}")
    
unique_keys = set(keys)
[x for x in unique_keys if x.endswith('pt') or x.endswith('tau')]